# Tarea PySpark

### Objetivo:
 Analizar la eficiencia de los jugadores en términos generales y por posición, así como determinar la contribución al equipo por jugador tomando en cuenta los datos obtenidos

Usarás la base de datos del archivo 'fusbol.csv' para obtener tus datos. Checa la estructura del archivo para ver si es necesario limpiar la informacion, ver su estructura y así sea más fácil completar la tarea. Besos.

### Ejercicio 1:
Carga la base de datos en un DataFrame de Pyspark (con 2 nucleos). Valida los rangos de los valores donde sea aplicable, así como su corrección (en caso de ser necesaria). 
Después, utilizando las variables más relevantes como 'Ast/90', 'PassCmp%', etc., concluye qué ligas tienen los mejores jugadores por posición. Es decir, si los mejores jugadores defensas son de la liga francesa, inglesa, etc., por ejemplo.
Como cada persona tiene una definición de "mejor", utiliza las siguientes metricas por posición:
- Delanteros: npG+A/90 y npxG+xA/90
- Medios: KeyPass/90 y PassCmp%
- Defensas: PressSucc% y Interceptions/90

Con los resultados obtenidos, grafica por posición para que tu conclusión tenga un respaldo visual también.

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, explode, split

In [2]:
spark = SparkSession.builder.appName('firstSession')\
    .config('spark.master', 'local[2]')\
    .config('spark.executor.memory', '1g')\
    .config("spark.sql.shuffle.partitions", 10)\
    .config('spark.driver.memory','1g')\
    .getOrCreate()

spark

In [3]:
# Cargo los datos y remuevo los nulls para tener un análisis robusto

data = spark.read.csv('fusbol.csv', header=True, inferSchema=True)
data = data.dropna()
data.printSchema()

root
 |-- Player: string (nullable = true)
 |-- Nation: string (nullable = true)
 |-- Pos: string (nullable = true)
 |-- Squad: string (nullable = true)
 |-- Comp: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Born: double (nullable = true)
 |-- MP: integer (nullable = true)
 |-- Starts: integer (nullable = true)
 |-- Min: integer (nullable = true)
 |-- 90s: double (nullable = true)
 |-- Ast/90: double (nullable = true)
 |-- npG/90: double (nullable = true)
 |-- npG+A/90: double (nullable = true)
 |-- xA/90: double (nullable = true)
 |-- npxG/90: double (nullable = true)
 |-- npxG+xA/90: double (nullable = true)
 |-- Shots/90: double (nullable = true)
 |-- SoTs/90: double (nullable = true)
 |-- SoT%: double (nullable = true)
 |-- Gls/Shot: double (nullable = true)
 |-- Gls/SoT: double (nullable = true)
 |-- AvgShotDist: double (nullable = true)
 |-- FKSht/90: double (nullable = true)
 |-- npxG/Shot: double (nullable = true)
 |-- np:G-xG: double (nullable = true)
 |--

In [4]:
# Valido que las métricas porcentuales estén entre 0 y 100

data = data.withColumn('PassCmp%', when(col('PassCmp%') < 0, 0).otherwise(col('PassCmp%')))
data = data.withColumn('PassCmp%', when(col('PassCmp%') > 100, 100).otherwise(col('PassCmp%')))
data = data.withColumn('PressSucc%', when(col('PressSucc%') < 0, 0).otherwise(col('PressSucc%')))
data = data.withColumn('PressSucc%', when(col('PressSucc%') > 100, 100).otherwise(col('PressSucc%')))


# Para las demás, simplemente corroboro que sean valores positivos

data = data.withColumn('npG+A/90', when(col('npG+A/90') < 0, 0).otherwise(col('npG+A/90')))
data = data.withColumn('npxG+xA/90', when(col('npxG+xA/90') < 0, 0).otherwise(col('npxG+xA/90')))
data = data.withColumn('KeyPass/90', when(col('KeyPass/90') < 0, 0).otherwise(col('KeyPass/90')))
data = data.withColumn('Interceptions/90', when(col('Interceptions/90') < 0, 0).otherwise(col('Interceptions/90')))

In [5]:
# Ahora haré que los jugadores con más de una posición se repitan (variando las posiciones)

data = data.withColumn("UniquePos", explode(split(col("Pos"), ","))).drop("Pos")

# Así los jugadores podrán aparecer en más de una categoría al clasificarlos

delanteros = data.filter(col("UniquePos") == "FW")
medios = data.filter(col("UniquePos") == "MF")
defensas = data.filter(col("UniquePos") == "DF")

In [ ]:
# Agrupamos por liga y calculamos los promedios pertinentes
from pyspark.sql.functions import avg

resultados_delanteros = delanteros.groupBy('Comp').agg(avg('npG+A/90').alias('Promedio_npG_A_90'), avg('npxG+xA/90').alias('Promedio_npxG_xA_90')).sort(col('Promedio_npG_A_90').desc())
resultados_medios = medios.groupBy('Comp').agg(avg('KeyPass/90').alias('Promedio_KeyPass_90'), avg('PassCmp%').alias('Promedio_PassCmp')).sort(col('Promedio_KeyPass_90').desc())
resultados_defensas = defensas.groupBy('Comp').agg(avg('PressSucc%').alias('Promedio_PressSucc'), avg('Interceptions/90').alias('Promedio_Interceptions_90')).sort(col('Promedio_PressSucc').desc())

resultados_delanteros.show()
resultados_medios.show()
resultados_defensas.show()

+------------------+------------------+-------------------+
|              Comp| Promedio_npG_A_90|Promedio_npxG_xA_90|
+------------------+------------------+-------------------+
|        it Serie A|0.4549264705882352| 0.4380882352941176|
|     de Bundesliga|0.4301986754966887|   0.41476821192053|
|        fr Ligue 1| 0.375688622754491| 0.3823353293413174|
|eng Premier League|0.3722142857142859| 0.3900000000000001|
|        es La Liga|0.3259375000000001|0.34431249999999997|
+------------------+------------------+-------------------+



Vemos que la Serie A tiene a los mejores delanteros y medios. En cuanto a defensas, el lugar se disputa entre la Ligue 1 y la Bundesliga ya que ambas tienen el primer lugar en una métrica y el segundo lugar en la otra.

In [ ]:
# Graficamos los resultados
import matplotlib.pyplot as plt
import numpy as np

# Datos de las ligas
ligas = resultados_delanteros.select('Comp').rdd.flatMap(lambda x: x).collect()

# Promedios de métricas para delanteros por liga
promedios_delanteros_npGA90 = resultados_delanteros.select('Promedio_npG_A_90').rdd.flatMap(lambda x: x).collect()
promedios_delanteros_npxGxA90 = resultados_delanteros.select('Promedio_npxG_xA_90').rdd.flatMap(lambda x: x).collect()

# Promedios de métricas para medios por liga
promedios_medios_keyPass90 = resultados_medios.select('Promedio_KeyPass_90').rdd.flatMap(lambda x: x).collect()
promedios_medios_passCmp = resultados_medios.select('Promedio_PassCmp').rdd.flatMap(lambda x: x).collect()

# Promedios de métricas para defensas por liga
promedios_defensas_pressSucc = resultados_defensas.select('Promedio_PressSucc').rdd.flatMap(lambda x: x).collect()
promedios_defensas_interceptions90 = resultados_defensas.select('Promedio_Interceptions_90').rdd.flatMap(lambda x: x).collect()

# Configuración de las barras
bar_width = 0.35
index = np.arange(len(ligas))

# Graficar para delanteros
plt.figure(figsize=(10, 6))
plt.bar(index, promedios_delanteros_npGA90, bar_width, label='Promedio npG+A/90', color='skyblue')
plt.bar(index + bar_width, promedios_delanteros_npxGxA90, bar_width, label='Promedio npxG+xA/90', color='lightgreen')
plt.xlabel('Ligas')
plt.ylabel('Promedio')
plt.title('Promedio de Métricas para Delanteros por Liga')
plt.xticks(index + bar_width / 2, ligas)
plt.legend()
plt.tight_layout()
plt.show()

# Graficar para medios
plt.figure(figsize=(10, 6))
plt.bar(index, promedios_medios_keyPass90, bar_width, label='Promedio KeyPass/90', color='skyblue')
plt.bar(index + bar_width, promedios_medios_passCmp, bar_width, label='Promedio PassCmp%', color='lightgreen')
plt.xlabel('Ligas')
plt.ylabel('Promedio')
plt.title('Promedio de Métricas para Medios por Liga')
plt.xticks(index + bar_width / 2, ligas)
plt.legend()
plt.tight_layout()
plt.show()

# Graficar para defensas
plt.figure(figsize=(10, 6))
plt.bar(index, promedios_defensas_pressSucc, bar_width, label='Promedio PressSucc%', color='skyblue')
plt.bar(index + bar_width, promedios_defensas_interceptions90, bar_width, label='Promedio Interceptions/90', color='lightgreen')
plt.xlabel('Ligas')
plt.ylabel('Promedio')
plt.title('Promedio de Métricas para Defensas por Liga')
plt.xticks(index + bar_width / 2, ligas)
plt.legend()
plt.tight_layout()
plt.show()

### Ejercicio 2:

Ahora hagamos algo un poco más interesante. Escoge algún jugador de todos los disponibles y toma 5 metricas, las que quieras. Debes concluir en qué percentil se encuentra el jugador en esas métricas que escogiste. Obviamente, vas a comparar sus valores con todos los demás con los que comparte posición y liga, para no tener un sesgo y que la información no pierda robustez. Por último, genera un DataFrame de Pyspark con todos los datos solicitados. Muestra el DataFrame y conviertelo a otro de tipo pandas. Muestra los dos.

In [ ]:
# Vamos a analizar Mohamed Salah. Primero filtramos por liga a los delanteros
delanteros_premier = delanteros.filter(delanteros['Comp'] == 'eng Premier League')

# Guardamos la información de Salah
metricas = ['npG+A/90', 'npxG+xA/90', 'KeyPass/90', 'Shots/90', 'SoTs/90']
salah_data = delanteros_premier.filter(col('Player') == 'Mohamed Salah')
salah_metricas = [salah_data.select(metrica).collect()[0][0] for metrica in metricas]

print(salah_metricas)

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

#Guardamos los percentiles por métrica
percentiles_df = delanteros_premier.approxQuantile(metricas, [i * 0.01 for i in range(1, 101)], relativeError=0.0)

In [ ]:
#Comparamos con las de Salah e imprimimos
salah_percentiles = [
    next((i / 100 for i, p in enumerate(metric_percentiles) if salah_metric_value <= p), 1.0)
    for metric_percentiles, salah_metric_value in zip(percentiles_df, salah_metricas)
]

salah_percentiles

In [ ]:
# Finalmente calculamos el promedio de estos percentiles para situar a Mohamed Salah

print('Situamos a Salah en el percentil', sum(salah_percentiles)/5, 'dadas las métricas que elegimos')